# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,flying fish cove,-10.4217,105.6791,26.82,89,75,3.09,CX,1705924468
1,1,bardai,11.6500,7.2333,31.46,5,3,6.46,NG,1705924469
2,2,olinda,-8.0089,-34.8553,28.85,74,40,2.06,BR,1705924470
3,3,port-aux-francais,-49.3500,70.2167,6.34,81,93,11.78,TF,1705924203
4,4,longyearbyen,78.2186,15.6401,-9.09,67,20,10.29,SJ,1705924471


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)
# Display the map
city_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_city_data_df = city_data_df[(city_data_df['Cloudiness'] == 0) & \
                                   (city_data_df['Max Temp'] >= 21) & \
                                   (city_data_df['Max Temp'] <= 27) & \
                                   (city_data_df['Wind Speed'] <= 4.5)]

# Drop any rows with null values
narrow_city_data_df = narrow_city_data_df.dropna(how='any')

# Display sample data
narrow_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
28,28,junin de los andes,-39.9504,-71.0694,23.96,25,0,0.74,AR,1705924486
98,98,arish,31.1316,33.7984,22.10,53,0,3.60,EG,1705924526
143,143,puerto carreno,6.1890,-67.4859,24.99,78,0,4.12,CO,1705924552
150,150,agat village,13.3886,144.6585,25.06,89,0,2.57,GU,1705924556
261,261,pedregal,8.3667,-82.4333,25.05,88,0,1.03,PA,1705924621
412,412,al jahra',29.3375,47.6581,22.53,20,0,4.16,KW,1705924712


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_city_data_df[['City' , 'Country' , 'Lat' , 'Lng' ,'Humidity']]
hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = " "

# Display sample data
hotel_df

C:\Users\royip\AppData\Local\Temp\ipykernel_4536\576075356.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = " "


,City,Country,Lat,Lng,Humidity,Hotel Name
28,junin de los andes,AR,-39.9504,-71.0694,25,
98,arish,EG,31.1316,33.7984,53,
143,puerto carreno,CO,6.1890,-67.4859,78,
150,agat village,GU,13.3886,144.6585,89,
261,pedregal,PA,8.3667,-82.4333,88,
412,al jahra',KW,29.3375,47.6581,20,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"

params = {
    "apiKey":geoapify_key,
    "categories":categories	
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
junin de los andes - nearest hotel: Apart Hotel "Pilmaiquén"
arish - nearest hotel: سينا
puerto carreno - nearest hotel: Carreño Plaza
agat village - nearest hotel: Navy Getway Inns & Suites
pedregal - nearest hotel: Aranjuez Hotel & Suites
al jahra' - nearest hotel: Al Jahra Copthorne Hotel & Resort


,City,Country,Lat,Lng,Humidity,Hotel Name
28,junin de los andes,AR,-39.9504,-71.0694,25,"Apart Hotel ""Pilmaiquén"""
98,arish,EG,31.1316,33.7984,53,سينا
143,puerto carreno,CO,6.1890,-67.4859,78,Carreño Plaza
150,agat village,GU,13.3886,144.6585,89,Navy Getway Inns & Suites
261,pedregal,PA,8.3667,-82.4333,88,Aranjuez Hotel & Suites
412,al jahra',KW,29.3375,47.6581,20,Al Jahra Copthorne Hotel & Resort


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
city_hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols=["Hotel Name" , "Country"]
)

# Display the map
city_hotel_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)